In [ ]:
import pandas as pd
import cx_Oracle
import datetime

# CSV 파일 경로
csv_file = '저장할공간'

# Oracle 데이터베이스 연결 설정
dsn_tns = cx_Oracle.makedsn('DB URL', 'PORT번호', service_name='XE')
conn = cx_Oracle.connect(user='DB사용자이름', password='DB PASSWORD', dsn=dsn_tns)
cursor = conn.cursor()

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 결측치 처리
df = df.where(pd.notnull(df), '')

# int64 타입을 object 타입으로 변환
df['org_code'] = df['org_code'].astype(str)
df['application_repeat_code'] = df['application_repeat_code'].astype(str)
df['policy_field_code'] = df['policy_field_code'].astype(str)

# CREATED_AT 및 UPDATED_AT 열 추가
current_time = datetime.datetime.now()
df['CREATED_AT'] = current_time
df['UPDATED_AT'] = current_time

# 데이터베이스에 삽입할 SQL 쿼리
insert_sql = '''
INSERT INTO 정책테이블 (
    policy_id, org_code, policy_name, policy_desc, support_content, support_scale, operation_period, 
    application_repeat_code, application_period, age_info, residence_income_condition, application_process, 
    participation_limit_target, application_procedure, main_department_name, main_department_contact, 
    main_department_phone, operating_institution_name, operating_institution_contact, operating_institution_phone, 
    submission_documents, evaluation_and_announcement, application_site_url, reference_site_url1, reference_site_url2, 
    etc, policy_field_code, CREATED_AT, UPDATED_AT
) VALUES (
    :1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :19, :20, :21, :22, :23, :24, 
    :25, :26, :27, :28, :29
)
'''

# DataFrame의 각 행을 데이터베이스에 삽입
for idx, row in df.iterrows():
    row_data = (
        row['policy_id'], row['org_code'], row['policy_name'], row['policy_desc'], row['support_content'],
        row['support_scale'], row['operation_period'], row['application_repeat_code'], row['application_period'],
        row['age_info'], row['residence_income_condition'], row['application_process'], row['participation_limit_target'],
        row['application_procedure'], row['main_department_name'], row['main_department_contact'], row['main_department_phone'],
        row['operating_institution_name'], row['operating_institution_contact'], row['operating_institution_phone'],
        row['submission_documents'], row['evaluation_and_announcement'], row['application_site_url'],
        row['reference_site_url1'], row['reference_site_url2'], row['etc'], row['policy_field_code'],
        row['CREATED_AT'], row['UPDATED_AT']
    )
    try:
        cursor.execute(insert_sql, row_data)
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        print(f"Error inserting row {idx+1}: {error.message}")
        print("Values being inserted:")
        for i, val in enumerate(row_data):
            print(f"{i+1}: {val}")

# 커밋하고 연결 종료
conn.commit()
cursor.close()
conn.close()

print("데이터추가완료")
